In [2]:
from scapy.all import *
from netml.pparser.parser import PCAP
import pandas as pd
import glob
import numpy as np

In [ ]:
files = glob.glob("csv/*.csv")
for i in range(len(files)):
    files[i] = files[i][4:-4]

for file in files:
    print('processing '+file)
    csv = pd.read_csv('csv/'+file+'.csv').to_numpy()
    csv = np.append(csv,[[0]]*len(csv), axis=1)
    
    netml = PCAP('pcap/'+file+'.pcap')
    netml.pcap2pandas()
    datetime = netml.df.to_numpy()
    for i in range(len(csv)):
        [date,time] = str(datetime[i,0]).split(' ')
        csv[i,0] = date
        csv[i,1] = time
    
    scapydf = rdpcap('pcap/'+file+'.pcap')
    for i in range(len(csv)):
        if csv[i,15] == 'Modbus/TCP':
            if Raw in scapydf[i]:
                csv[i,-1] = scapydf[i][Raw].load[2:].hex()
            else :
                csv[i,-1] = 0
        else :
            csv[i,-1] = 0
    
    pd.DataFrame(csv).to_csv('output/'+file+'.csv', index=False)

In [ ]:
# "2023-02-16"
dayone = [["10:45:00","11:16:00"],
          ["12:35:00","13:06:00"],
          ["13:45:00","14:16:00"],
          ["14:55:00","15:26:00"],
          ["20:00:00","20:31:00"]]
# "2023-02-17"
daytwo = [["10:05:00","10:21:00"],
          ["10:55:00","11:11:00"],
          ["11:45:00","12:01:00"],
          ["12:35:00","12:51:00"],
          ["13:25:00","13:41:00"],
          ["14:15:00","14:46:00"]]

output = np.array([['date','time','eth.src','eth.dst','eth.type','ip.src',
                    'ip.dst','ip.version','ip.hdr_len','ip.dsfield','ip.len',
                    'ip.id','ip.flags','ip.frag_offset','ip.ttl','ip.proto','tcp.srcport',
                    'tcp.dstport','tcp.seq','tcp.ack','tcp.hdr_len','tcp.flags','tcp.window_size',
                    'tcp.time_delta','tcp.time_relative','raw','attack']])

attack = 0

files = glob.glob("output/*.csv")
for i in range(len(files)):
    files[i] = files[i][7:-4]

for file in files:
    print('processing '+file)
    csv = pd.read_csv('output/'+file+'.csv').to_numpy()
    csv = np.append(csv,[[0]]*len(csv), axis=1)
    for row in csv:
        if row[0] == "2023-02-16":
            for time in dayone:
                if row[1] >= time[0] and row[1] <= time[1]:
                    row[-1] = 1
                    attack+=1
        elif row[0] == "2023-02-17":
             for time in daytwo:
                if row[1] >= time[0] and row[1] <= time[1]:
                    row[-1] = 1  
                    attack+=1
        output = np.append(output,[row],axis = 0)

processing pn-bs-ms
processing synflood


In [ ]:
print(attack)
output = output[np.lexsort((output[:,0], output[:,1]))]
to_file = pd.DataFrame(output)
to_file.to_csv('output.csv', index=False)